In [2]:
import spacy
nlp = spacy.load('en_core_web_sm') # load the english language library (load a model called nlp)

In [12]:
def tokenize(doc):
    for token in doc:
        print(token.text,token.pos_,token.dep_) # token attributes (e.g. the part of speech)

# reads in the unicode string and parses the sentence into tokens (individual words)
doc = nlp(u'My name is Brandon and I have to write a card for my birthday.') # doc holds the processed text
# nlp() -> the text enters a "processing pipeline," which breaks down the text and conducts a series of operations on it

# tokenization:
doc2 = nlp(u"Tesla's stock is dropping. Oh the horror!")
tokenize(doc2)
# can access specific tokens in the doc object with indexing (doc2[0])

# we can slice the processed string and take certain tokens (e.g. doc[0:3])
# spacy will convert this to a Span object because we are taking a range of the tokens

# spacy can automatically seperate the string based on sentences
for sentence in doc2.sents:
    print(sentence)

Tesla PROPN poss
's PART case
stock NOUN nsubj
is VERB aux
dropping VERB ROOT
. PUNCT punct
Oh INTJ intj
the DET det
horror NOUN ROOT
! PUNCT punct
Tesla's stock is dropping.
Oh the horror!


In [ ]:
# tokenization is breaking up the sentence into smaller components (e.g. words)
# this is the fundamental process for understanding the sentence
# we split by whitespace, then remove characters in the beginning/end, 